Proof of <a class="ProveItLink" href="../../../../../../_theory_nbs_/theory.ipynb">proveit</a>.<a class="ProveItLink" href="../../../../../_theory_nbs_/theory.ipynb">physics</a>.<a class="ProveItLink" href="../../../../_theory_nbs_/theory.ipynb">quantum</a>.<a class="ProveItLink" href="../../theory.ipynb">QPE</a>.<a class="ProveItLink" href="../../theorems.ipynb#_best_guarantee_delta_nonzero">_best_guarantee_delta_nonzero</a> theorem
========

In [1]:
import proveit
theory = proveit.Theory() # the theorem's theory
from proveit import b, m, theta, defaults
from proveit.logic import NotEquals, InSet
from proveit.numbers import (
        zero, one, two, pi, Abs, Add, Complex, frac, IntervalOC, Less, greater, Mult, Exp)
# QPE common exprs
from proveit.physics.quantum.QPE import _b_round, _delta_b_round, _t, _two_pow_t
 # QPE theorems
from proveit.physics.quantum.QPE import (
        _alpha_m_mod_as_geometric_sum, _alpha_are_complex, _best_round_is_int,
        _delta_b_is_real, _delta_b_is_zero_or_non_int,
        _phase_from_best_with_delta_b, _scaled_delta_b_round_in_interval,
        _t_in_natural_pos, _two_pow_t_is_nat_pos)
from proveit.trigonometry import Sin

In [2]:
%proving _best_guarantee_delta_nonzero

In [3]:
defaults.assumptions = [_best_guarantee_delta_nonzero.antecedent]

Keep the factor of $2$ from combining with $2^t$ in the steps below:

In [4]:
Exp.change_simplification_directives(factor_numeric_rational=True)

In [5]:
alpha_index = _best_guarantee_delta_nonzero.consequent.lhs.base.operand.index

## Start from $\alpha_{m~\textrm{mod}~2^t}$ expressed as a geometric sum, instantiated with $m:b_r$

In [6]:
_alpha_m_mod_as_geometric_sum

|- forall_{m in Integer} (alpha_{m mod 2^{t}} = ((1 / 2^{t}) * (Sum_{k = 0}^{2^{t} - 1} (e^{2 * pi * i * (phase - (m / 2^{t}))})^{k})))

In [7]:
_alpha_eq_01 = (
        _alpha_m_mod_as_geometric_sum.instantiate({m: _b_round}))

### Put this in terms of $\delta_{b_r}$

In [8]:
_phase_from_best_with_delta_b

|- forall_{b in Integer} (phase = ((b / 2^{t}) + delta_{b}))

In [9]:
_phase_from_best_with_delta_b_inst = (
        _phase_from_best_with_delta_b.instantiate({b: _b_round}))

In [10]:
# We want to know that _delta_b_round and 2^t are real so that
# auto-simplification will manifest in the substitute() step further below
_delta_b_is_real

|- forall_{b in Integer} (delta_{b} in Real)

In [11]:
_delta_b_is_real.instantiate({b: _b_round})

|- delta_{b_{r}} in Real

In [12]:
with Exp.temporary_simplification_directives() as tmp_directives:
    tmp_directives.reduce_double_exponent = False # don't simplify k double-exponent
    _alpha_eq_02 = (
        _alpha_eq_01.inner_expr().rhs.factors[1].summand.base.exponent.
        factors[3].operands[0].substitute(_phase_from_best_with_delta_b_inst.rhs))

## Evaluate the Finite Geometric Sum

To evaluate the summation as a finite geometric sum, we need to establish that the constant ratio $e^{2\pi i \delta_{b_r}} \neq 1$ which is automatically proven given $\delta_{b_r} \notin \mathbb{Z}$.

In [13]:
_delta_b_is_zero_or_non_int

|- forall_{b in {b_{f}, b_{r}}} ((delta_{b} = 0) or (delta_{b} not-in Integer))

In [14]:
_delta_b_is_zero_or_non_int_inst = _delta_b_is_zero_or_non_int.instantiate({b: _b_round})

In [15]:
_delta_b_round_non_int = _delta_b_is_zero_or_non_int_inst.derive_right_if_not_left()

We can then evaluate the sum using the standard formula for a finite geometric sum:

In [16]:
_alpha_eq_03 = _alpha_eq_02.inner_expr().rhs.factors[1].geom_sum_reduce()

## Take the Magnitude (Abs) of Both Sides

### Pre-requisites to allow auto-simplification to go through

Establish that the $\alpha$ expression is in the Complex numbers.

In [17]:
_alpha_are_complex.instantiate({m:alpha_index})

|- alpha_{b_{r} mod 2^{t}} in Complex

Prove that $\pi |\delta_{b_r}| \in \left(0, \frac{\pi}{2}\right)$ which is important to ensure the denominator is non-zero.  Along the way, we prove $2^t \pi |\delta_{b_r}| \leq \frac{\pi}{2}$ which will be useful for bounding the numerator.

In [18]:
angle = Mult(pi, Abs(_delta_b_round))

In [19]:
Less(zero, angle).prove()

{delta_{b_{r}} != 0} |- 0 < (pi * |delta_{b_{r}}|)

In [20]:
scaled_abs_bound = (Abs(_scaled_delta_b_round_in_interval.element).deduce_weak_upper_bound(frac(one, two))
         .inner_expr().lhs.simplify())

Close enough to $2^t \pi |\delta_{b_r}| \leq \frac{\pi}{2}$ (the translation occurs automatically via canonical forms):

In [21]:
two_pow_t_times_angle__bound = scaled_abs_bound.left_mult_both_sides(pi)

In [22]:
two_pow_t_bound = _two_pow_t.deduce_bound(_t_in_natural_pos.derive_element_lower_bound())

In [23]:
two_pow_t_times_angle__bound.lhs.deduce_bound(greater(_two_pow_t, one))

{delta_{b_{r}} != 0} |- (pi * 2^{t} * |delta_{b_{r}}|) > (pi * |delta_{b_{r}}|)

In [24]:
Less(angle, frac(pi, two)).prove()

{delta_{b_{r}} != 0} |- (pi * |delta_{b_{r}}|) < (pi / 2)

### Now take the absolute value on each side 

In [25]:
with Mult.temporary_simplification_directives() as tmp_directives:
    # pull the 2^t to the front
    tmp_directives.order_key_fn = lambda factor : 0 if factor==_two_pow_t else 1
    _alpha_eq_04 = _alpha_eq_03.abs_both_sides()

## Bound the numerator and the denominator

### Bound the sin function in the _denominator_ utilizing $\sin{\theta} < \theta$ for $\theta > 0$

In [26]:
denominator_sin_bound = _alpha_eq_04.rhs.factors[1].denominator.deduce_linear_upper_bound()

### Bound the sin function in the _numerator_ utilizing $\sin{\theta} \ge \frac{2}{\pi}(\theta)$ for $0 < \theta \le \frac{\pi}{2}$

In [27]:
numerator_sin_bound = _alpha_eq_04.rhs.factors[1].numerator.deduce_linear_lower_bound()

### Apply these numerator and denominator bounds

In [28]:
rhs_bound = _alpha_eq_04.rhs.deduce_bound([numerator_sin_bound, denominator_sin_bound])

In [29]:
_alpha_ineq = _alpha_eq_04.apply_transitivity(rhs_bound)

## Finally square both sides

In [30]:
_alpha_ineq.square_both_sides()

{delta_{b_{r}} != 0} |- |alpha_{b_{r} mod 2^{t}}|^{2} > (2 / pi)^{2}

_best_guarantee_delta_nonzero may now be readily provable (assuming required theorems are usable).  Simply execute "%qed".


In [31]:
%qed

proveit.physics.quantum.QPE._best_guarantee_delta_nonzero has been proven.


step type	requirements	statement
0	deduction	1	|- (delta_{b_{r}} != 0) => (|alpha_{b_{r} mod 2^{t}}|^{2} > (4 / pi^{2}))
1	instantiation	424, 2, 3	{delta_{b_{r}} != 0} |- (4 / pi^{2}) < |alpha_{b_{r} mod 2^{t}}|^{2}
	P(_a) : _a < |alpha_{b_{r} mod 2^{t}}|^{2}, x : 4 / pi^{2}, y : (2 / pi)^{2}
2	instantiation	4, 622, 5, 6, 7, 8	{delta_{b_{r}} != 0} |- (2 / pi)^{2} < |alpha_{b_{r} mod 2^{t}}|^{2}
	a : 2, x : 2 / pi, y : |alpha_{b_{r} mod 2^{t}}|
3	instantiation	466, 9, 10, 11	|- (4 / pi^{2}) = (2 / pi)^{2}
	a : 4 / pi^{2}, b : 4 * pi^{-2}, c : 4 * pi^{-2}, d : (2 / pi)^{2}
4	conjecture		|- forall_{a, x, y in Real | 0 <= x < y, a > 0} (x^{a} < y^{a})
	proveit.numbers.exponentiation.exp_pos_less
5	instantiation	632, 538, 29	|- (2 / pi) in Real
	A : RealPos, B : Real, x : 2 / pi
6	instantiation	632, 333, 12	|- |alpha_{b_{r} mod 2^{t}}| in Real
	A : RealNonNeg, B : Real, x : |alpha_{b_{r} mod 2^{t}}|
7	instantiation	337, 13, 14	{delta_{b_{r}} != 0} |- (0 <= (2 / pi)) and ((2 / pi) < |alpha_{b_{r} mod 2^{t}}|)
	A : 0 <= (2 / pi), B : (2 / pi) < |alpha_{b_{r} mod 2^{t}}|
8	instantiation	572, 627	|- 2 > 0
	n : 2
9	instantiation	280, 553, 15, 16, 17*	|- (4 / pi^{2}) = (4 * pi^{-2})
	x : 4, y : pi^{2}
10	instantiation	566	|- (4 * pi^{-2}) = (4 * pi^{-2})
	x : 4 * pi^{-2}
11	instantiation	409, 18	|- (4 * pi^{-2}) = (2 / pi)^{2}
	x : (2 / pi)^{2}, y : 4 * pi^{-2}
12	instantiation	19, 35	|- |alpha_{b_{r} mod 2^{t}}| in RealNonNeg
	a : alpha_{b_{r} mod 2^{t}}
13	instantiation	20, 21	|- 0 <= (2 / pi)
	x : 2 / pi
14	instantiation	424, 22, 23	{delta_{b_{r}} != 0} |- |alpha_{b_{r} mod 2^{t}}| > (2 / pi)
	P(_a) : _a > (2 / pi), x : |alpha_{b_{r} mod 2^{t}}|, y : (1 / 2^{t}) * (sin(2^{t} * pi * |delta_{b_{r}}|) / sin(pi * |delta_{b_{r}}|))
15	instantiation	632, 621, 24	|- pi^{2} in Complex
	A : Real, B : Complex, x : pi^{2}
16	instantiation	25, 479, 612, 420	|- pi^{2} != 0
	a : pi, b : 2
17	instantiation	62, 479, 622, 564, 420, 26*	|- (pi^{2})^{-1} = pi^{-2}
	a : pi, b : 2, c : -1
18	instantiation	573, 27, 28	|- (2 / pi)^{2} = (4 * pi^{-2})
	x : (2 / pi)^{2}, y : (2 * pi^{-1})^{2}, z : 4 * pi^{-2}
19	conjecture		|- forall_{a in Complex} (|a| in RealNonNeg)
	proveit.numbers.absolute_value.abs_complex_closure
20	conjecture		|- forall_{x in RealNonNeg} (x >= 0)
	proveit.numbers.number_sets.real_numbers.nonneg_if_in_real_nonneg
21	instantiation	632, 379, 29	|- (2 / pi) in RealNonNeg
	A : RealPos, B : RealNonNeg, x : 2 / pi
22	instantiation	404, 84, 30, 31, 32, 117, 33*	{delta_{b_{r}} != 0} |- (2 / pi) < ((1 / 2^{t}) * (sin(2^{t} * pi * |delta_{b_{r}}|) / sin(pi * |delta_{b_{r}}|)))
	a : 1 / 2^{t}, x : (2 * 2^{t} * |delta_{b_{r}}|) / (pi * |delta_{b_{r}}|), y : sin(2^{t} * pi * |delta_{b_{r}}|) / sin(pi * |delta_{b_{r}}|)
23	instantiation	34, 35, 36, 37, 38*	{delta_{b_{r}} != 0} |- |alpha_{b_{r} mod 2^{t}}| = ((1 / 2^{t}) * (sin(2^{t} * pi * |delta_{b_{r}}|) / sin(pi * |delta_{b_{r}}|)))
	x : alpha_{b_{r} mod 2^{t}}, y : (1 / 2^{t}) * ((1 - e^{2 * pi * i * delta_{b_{r}} * 2^{t}}) / (1 - e^{2 * pi * i * delta_{b_{r}}}))
24	instantiation	39, 508, 634	|- pi^{2} in Real
	a : pi, b : 2
25	conjecture		|- forall_{a, b in Complex | a != 0} (a^{b} != 0)
	proveit.numbers.exponentiation.exp_not_eq_zero
26	instantiation	40, 612, 541, 598*	|- (2 * (-1)) = (-2)
	x : 2, y : 1
27	instantiation	582, 41	|- (2 / pi)^{2} = (2 * pi^{-1})^{2}
	f(_a) : _a^{2}, x : 2 / pi, y : 2 * pi^{-1}
28	instantiation	42, 612, 417, 627, 43*, 44*	|- (2 * pi^{-1})^{2} = (4 * pi^{-2})
	a : 2, b : pi^{-1}, n : 2
29	instantiation	45, 299, 537, 420	|- (2 / pi) in RealPos
	a : 2, b : pi
30	instantiation	46, 245, 318, 190	{delta_{b_{r}} != 0} |- ((2 * 2^{t} * |delta_{b_{r}}|) / (pi * |delta_{b_{r}}|)) in Real
	a : 2 * 2^{t} * |delta_{b_{r}}|, b : pi * |delta_{b_{r}}|
31	instantiation	46, 183, 241, 182	{delta_{b_{r}} != 0} |- (sin(2^{t} * pi * |delta_{b_{r}}|) / sin(pi * |delta_{b_{r}}|)) in Real
	a : sin(2^{t} * pi * |delta_{b_{r}}|), b : sin(pi * |delta_{b_{r}}|)
32	instantiation	401, 47, 48	{de